# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [2]:
import requests
import re

url = 'https://en.wikipedia.org/wiki/Data_science'

In [22]:
wiki = requests.get(url).content

In [4]:
wiki[:200]

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Data science - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wg'

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [5]:
from bs4 import BeautifulSoup

In [6]:
soup = BeautifulSoup(wiki, 'lxml')
soup.text[:400]

'\n\n\nData science - Wikipedia\ndocument.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"db593634-089f-4946-93e0-174b0c01ebef","wgCSPNonce":!1,"wg'

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [7]:
url= 'http://wikipedia.org'
wiki = requests.get(url).content
soup = BeautifulSoup(wiki, 'html.parser')
a= soup.find_all('a')

In [8]:
a_links=[i['href'] for i in a]

In [9]:
links_clean=[i for i in a_links if '%' not in i]

In [10]:
abs_links=[i for i in a_links if 'http' in i]

In [11]:
re_links=['https:'+i for i in a_links if 'http' not in i]

In [12]:
b_links= abs_links + re_links
print(len(b_links))

325


In [13]:
print(len(set(b_links))) #Links únicos

314


In [14]:
b_links[:5]

['https://en.wikipedia.org/wiki/List_of_Wikipedia_mobile_applications',
 'https://creativecommons.org/licenses/by-sa/3.0/',
 'https://en.wikipedia.org/',
 'https://es.wikipedia.org/',
 'https://ja.wikipedia.org/']

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [15]:
import os

In [16]:
os.chdir('Wikipedia')

In [17]:
os.getcwd()

'/Users/estefhanymorenovega/datamex_102020/module-1/lab-parallelization/your-code/Wikipedia'

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [18]:
from slugify import slugify

In [29]:
n1=slugify('https://es.wikipedia.org/wiki/Historia_del_arte') +'.html'
n1

'https-es-wikipedia-org-wiki-historia-del-arte.html'

In [44]:
def index_page(link):
    url= link
    wiki = requests.get(url).content
    soup = BeautifulSoup(wiki, 'html.parser')
    name=slugify(link)+ '.html'
    
    archivo=open(name, 'w', encoding= 'UTF-8 ')
    archivo.write(soup.text)
    archivo.close()
    

In [45]:
index_page('https://es.wikipedia.org/wiki/Historia_del_arte')

In [46]:
index_page('https://es.wikipedia.org/wiki/Nutrici%C3%B3n')

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [48]:
prueba=b_links[:10]

In [49]:
%%time
for i in prueba:
    index_page(i)    

CPU times: user 978 ms, sys: 38.2 ms, total: 1.02 s
Wall time: 4.13 s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [50]:
import multiprocessing

In [54]:
%%time
pool = multiprocessing.Pool()
result = pool.map(index_page,prueba)
pool.terminate()
    

CPU times: user 14.6 ms, sys: 23.1 ms, total: 37.7 ms
Wall time: 2.11 s


**Conclusión:** Tardó la mitad del tiempo a si se hubiera ejecutado de manera normal